In [1]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, StringIndexer
from xgboost.spark import SparkXGBClassifier, SparkXGBClassifierModel
from pyspark.ml import PipelineModel
from pyspark.sql.types import *
from pyspark import keyword_only  ## < 2.0 -> pyspark.ml.util.keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from pyspark.sql.functions import col, unix_timestamp, when
import pandas as pd
from time import time
# from miniapps.utils import nice_time
# from miniapps.utils import create_date_time_str
from pyspark.ml.feature import IndexToString, StringIndexer
import os
import warnings
from datetime import date
from pyspark.ml.classification import GBTClassifier 

In [2]:
from pyspark.ml.classification import GBTClassifier 

In [3]:
CONFIG_SPARK = [
        # ("spark.driver.memory", '4g'),
        # ("spark.driver.cores", '2'),
        ("spark.executor.memory", '15g'),
        ("spark.executor.cores", '3'),
        ("spark.executor.instances", '5')
    ]

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import *
from time import time
import os

# os.environ['PYSPARK_PYTHON'] = './environment/bin/python'
s = time()
configs = CONFIG_SPARK
conf = SparkConf().setAll(configs)

spark = SparkSession \
    .builder \
    .appName("crosssla") \
    .config(conf=conf) \
    .getOrCreate()
print("done to get spark:")
print(spark)

23/12/18 12:39:01 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.1.27 instead (on interface eno1)
23/12/18 12:39:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/18 12:39:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/18 12:39:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


done to get spark:


In [5]:
data = spark.read.csv('data.csv', inferSchema=True, header=True)

In [6]:
data.count()

563337

In [7]:
list_pd_code_remove = ['12211', '12121', '11111', '13111']

In [8]:
data = data.filter(~col("pd_code").isin(list_pd_code_remove))

In [9]:
data.count()

355651

In [10]:
column_data_types = data.dtypes
columns = [data_type for column_name, data_type in column_data_types ]
print(set(columns))


{'double', 'int', 'date', 'string'}


In [11]:
LIST_DATA_STRING = []
LIST_DATA_DATE = []
column_data_types = data.dtypes
columns = [data_type for column_name, data_type in column_data_types ]
# non_numeric_columns = [column_name for column_name, data_type in column_data_types if data_type != 'int' and data_type != 'double'] 
for i in range(len(columns)):
    if columns[i] == 'string':
        LIST_DATA_STRING.append(column_data_types[i][0])
    elif columns[i] == 'date':
        LIST_DATA_DATE.append(column_data_types[i][0])

In [12]:
column_data_types = data.dtypes
LIST_DATA_NUMERIC = [column_name for column_name, data_type in column_data_types if column_name not in LIST_DATA_STRING and column_name not in LIST_DATA_DATE]

In [13]:
def date_to_numeric(data, input_Col):
    data = data.withColumn(input_Col, unix_timestamp(col(input_Col), "yyyy-MM-dd").cast("double"))
    return data


In [14]:
def remove_null_string(data, column_string):
    for column in column_string:
        data = data.withColumn(column, when(data[column].isNull(), "Unknown").otherwise(data[column]))
    return data

In [15]:
LABEL_COL = 'pd_code'
LABEL_OUTPUT = LABEL_COL + '_'
# FEAT = [
#  'NBR_TXN_DAY',
#  'NBR_TXN_DAY_T_2',
#  'AMT_TOT_CREDIT_TXN_T_2'
# ]
# FEAT = columns.remove(LIST_DATA_NUMERIC)
LIST_DATA_STRING.remove(LABEL_COL)
LIST_DATA_STRING.remove("FULL_NAME")
data = data.fillna(value={**{x: 0.0 for x in LIST_DATA_NUMERIC}})
for date in LIST_DATA_DATE:
    data = date_to_numeric(data, date)

data = remove_null_string(data, LIST_DATA_STRING)

# data = data.sample(0.01)



In [16]:
LIST_DATA_STRING

['GENDER',
 'MARITAL_STATUS',
 'SECTOR',
 'EKYC_LEVEL',
 'CHANNEL',
 'OVERDRAFT_LOAN_TENURE',
 'pd_name',
 'acc_no']

In [17]:
data.show()

23/12/18 12:39:13 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-----------+--------------------+------+--------------+---+------------+------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+---------

In [18]:
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid='keep') for column in LIST_DATA_STRING]
assembler_inputs = [indexer.getOutputCol() for indexer in indexers] + LIST_DATA_NUMERIC

labelIndexer = StringIndexer(inputCol=LABEL_COL, outputCol=LABEL_OUTPUT, handleInvalid='keep')
# indexer_df = labelIndexer.fit(data)
# data = indexer_df.transform(data)

assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
# data = assembler.transform(data)

In [19]:
train, test = data.randomSplit([0.8, 0.2], seed=123)


In [46]:
params_setting = {'max_depth': 5,
                  'learning_rate': 0.03,
                  'num_classes': int(train.select(LABEL_COL).distinct().count()),
                  'num_workers' : 1}


In [21]:
from xgboost.spark import SparkXGBClassifier
from pyspark.ml import Pipeline 
xgb = SparkXGBClassifier(label_col=LABEL_OUTPUT, features_col="features", **params_setting)
# stages = [labelIndexer, indexers, assembler, xgb]
stages = indexers + [labelIndexer] + [assembler, xgb]
pipeline = Pipeline(stages=stages)
ModelPipeline = pipeline.fit(train)
predTest = ModelPipeline.transform(test)

2023-12-17 23:35:28,361 INFO XGBoost-PySpark: _fit Running xgboost-2.0.2 on 1 workers with
	booster params: {'objective': 'multi:softprob', 'device': 'cpu', 'learning_rate': 0.03, 'max_depth': 5, 'num_classes': 140, 'num_class': 140, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
23/12/17 23:35:28 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB
[23:35:37] task 0 got new rank 0                                    (0 + 1) / 1]
/home/ubuntupc/phuong_workspace/Cross-Selling/env/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:35:40] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_classes" } are not used.

  warnings.warn(smsg, UserWarning)
2023-12-17 23:44:25,973 INFO XGBoost-PySpark: _fit Finished xgboost training!   


In [22]:
ModelPipeline.save('save_model')


23/12/17 23:44:29 WARN TaskSetManager: Stage 78 contains a task of very large size (16278 KiB). The maximum recommended task size is 1000 KiB.
23/12/17 23:44:30 WARN TaskSetManager: Stage 87 contains a task of very large size (8688 KiB). The maximum recommended task size is 1000 KiB.


In [20]:
from pyspark.ml import PipelineModel
ModelPipeline = PipelineModel.load('save_model')
predTest = ModelPipeline.transform(test)

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(labelCol="pd_code_", predictionCol="prediction")
accuracy = evaluator.evaluate(predTest)
print("Accuracy = %s" % (accuracy))
print("Test Error = %s" % (1.0 - accuracy))
truePositive = evaluator.evaluate(predTest, {evaluator.metricName: "truePositiveRateByLabel", evaluator.metricLabel: 1.0})
print("truePositive: ", truePositive)
f1_score = evaluator.evaluate(predTest, {evaluator.metricName: "f1"})
print("f1 score:", f1_score)


23/12/18 12:39:31 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB
2023-12-18 12:39:41,326 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:41,346 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:41,403 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:41,514 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:41,546 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:41,645 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


Accuracy = 0.9998611583361671
Test Error = 0.00013884166383293373


23/12/18 12:39:44 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB
2023-12-18 12:39:49,505 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:49,632 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:49,695 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:49,702 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:49,718 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:49,746 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


truePositive:  1.0


23/12/18 12:39:52 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB
2023-12-18 12:39:56,780 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:56,856 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:57,070 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:57,174 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:57,368 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:39:57,376 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


f1 score: 0.9998611583361671


In [22]:
@udf
def orderByColValue(colValues):
    return [x[1] for x in sorted(zip(colValues, SORTED_LIST), key=lambda x: x[0], reverse=True)][:3]
SORTED_LIST = ModelPipeline.stages[8].labels
predTest = predTest.withColumn("product", orderByColValue('probability'))

In [23]:
predTest.show()

23/12/18 12:40:01 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB


+-----------+-----------+--------------------+------+--------------+---+------------+-------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------

2023-12-18 12:40:03,634 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


In [24]:
predTest = predTest.select(LIST_DATA_STRING + LIST_DATA_NUMERIC + LIST_DATA_DATE + ["prediction", 'product', "probability"])

In [25]:
predTest.show()

23/12/18 12:40:50 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB


+------+--------------+-------+----------+-------+---------------------+--------------------+--------------------+-----------+---+------------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+-------------------

2023-12-18 12:40:52,979 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


In [26]:
df = predTest.toPandas()
df.to_csv('data_infer.csv', index=False)

23/12/18 12:41:06 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB
2023-12-18 12:41:14,117 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:41:14,132 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:41:14,153 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:41:14,231 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:41:14,301 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs
2023-12-18 12:41:14,312 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


In [49]:
predTest.show()

23/12/18 00:27:13 WARN DAGScheduler: Broadcasting large task binary with size 21.6 MiB
2023-12-18 00:27:16,754 INFO XGBoost-PySpark: predict_udf Do the inference on the CPUs


+-----------+-----------+--------------------+------+--------------+---+------------+-------+----------+-------+-----------+-----------+---------+---------------+---------------+--------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+------------------+-----------------+----------------------+---------------------+-----------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------------------+----------------------+--------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+------------------------+------------------------+-----------------------+------------------+------------------+----------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------

In [ ]:
df = predTest.toPandas()
df.to_csv('data_infer.csv', index=False)

In [35]:
bac_pd = spark.read.csv("bac_pd_dim.csv", header=True, inferSchema=True)


In [39]:
@udf
def orderByColValue(colValues):
    return [x[1] for x in sorted(zip(colValues, SORTED_LIST), key=lambda x: x[0], reverse=True)][:1][0]
SORTED_LIST = ModelPipeline.stages[8].labels
predTest = predTest.withColumn("product", orderByColValue('probability'))

In [ ]:
predTest.show()

In [ ]:

# result = predTest.join(bac_pd.select("PRODUCT_CODE", "CAP4"), predTest["product"] == bac_pd["PRODUCT_CODE"], "left_outer").select(predTest["product"], bac_pd["CAP4"])
merged_df = pd.merge(predTest, bac_pd, left_on='product', right_on='PRODUCT_CODE', how='inner')
result = merged_df[['product', 'CAP4']]



In [ ]:
result.show()